# Compare-7: distinct ICD count should be similar for consecutive timespans

Description: check if distinct ICD count are similar for consecutive timespans.

Starting Author: Amy Jin (amy@careset.com)

Date: July 23rd, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connect to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) icd:                    icd column
# --------------------------------------- Outputs: --------------------------------------
# 1) Test result:   distinct count of ICD codes.


def compare_7(db_name, table_name, icd):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # MySQL to compute distinct icd count
        query = ('''
                SELECT COUNT(DISTINCT {col1})
                FROM {db}.{t1};
        '''.format(db = db_name, t1 = table_name, col1 = icd))

        cur.execute(query)
        
        print ("The distinct {} count in {}.{} is:".format(icd, db_name, table_name) + '\n')        
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            #print ('\n')
            
        cur.close()
        connection.close()            

## Test Example

In [3]:
compare_7('_amy', 'test_data_good','icd10cm')

The distinct icd10cm count in _amy.test_data_good is:

2000


In [4]:
compare_7('_amy', 'test_data_bad1','icd_dgns_cd')

The distinct icd_dgns_cd count in _amy.test_data_bad1 is:

138


In [5]:
compare_7('_amy', 'test_data_bad2','icd_dgns_cd')

The distinct icd_dgns_cd count in _amy.test_data_bad2 is:

88


## Internal Data Test Example
#### 2017 Patient Count by Provider ZIP

In [5]:
compare_7('icd_national', 'icd_national_merged_RQ17' ,'icd_dgns_cd')

The distinct icd_dgns_cd count in icd_national.icd_national_merged_RQ17 is:

64818


#### 2016 Patient Count by Provider ZIP

In [6]:
compare_7('icd_national', 'icd_national_RQ16' ,'icd_dgns_cd')

The distinct icd_dgns_cd count in icd_national.icd_national_RQ16 is:

65645


#### RQ17R12_9 Patient Count by Provider ZIP

In [7]:
compare_7('icd_national', 'icd_national_RQ17R12_9' ,'icd_dgns_cd')

The distinct icd_dgns_cd count in icd_national.icd_national_RQ17R12_9 is:

64931
